# Two-way Fixed Effects

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
import gpboost as gpb
from patsy import dmatrix

In [3]:
path = '../../../../data/processed/df_filtered.xlsx'
df = pd.read_excel(path)
df['log_pop'] = np.log(df['populations'])
df.dropna(subset=['log_pop'], inplace=True)
df

,island_ids,years,region_codes,prefecture_codes,populations,treatment_group,treatment_group_mainland,year_bridges_opened,after_treated,year_connect_mainland,...,area_km2,distance_m,elementary_school,junior_high_school,high_school,hospital,population_change_rate,control_group,time_since_intervention,log_pop
5,67,1995,32202,32,426.0,1,1,1999,0,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,NaN,0,-4,6.054439
6,67,2000,32202,32,324.0,1,1,1999,1,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,-23.943662,0,1,5.780744
7,67,2005,32202,32,302.0,1,1,1999,1,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,-6.790123,0,6,5.710427
8,67,2010,32202,32,265.0,1,1,1999,1,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,-12.251656,0,11,5.579730
9,67,2015,32202,32,197.0,1,1,1999,1,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,-25.660377,0,16,5.283204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,226,2000,39424,39,398.0,1,1,1957,1,1957,...,0.56,110.0,NaN,NaN,NaN,NaN,-4.784689,0,43,5.986452
1896,226,2005,39424,39,365.0,1,1,1957,1,1957,...,0.56,110.0,NaN,NaN,NaN,NaN,-8.291457,0,48,5.899897
1897,226,2010,39424,39,343.0,1,1,1957,1,1957,...,0.56,110.0,NaN,NaN,NaN,NaN,-6.027397,0,53,5.837730
1898,226,2015,39424,39,284.0,1,1,1957,1,1957,...,0.56,110.0,NaN,NaN,NaN,NaN,-17.201166,0,58,5.648974


In [ ]:
# グループ変数を結合して単一の列にする
df['group'] = df['island_ids'].astype(str) + '_' + df['years'].astype(str)

model = mixedlm("log_pop ~ after_treated", df, groups=df["group"], re_formula="~years")
result = model.fit()

# 結果の表示
print(result.summary())

/home/keisei/data-analysis/bridge-straw-effects/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/keisei/data-analysis/bridge-straw-effects/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/home/keisei/data-analysis/bridge-straw-effects/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


           Mixed Linear Model Regression Results
Model:               MixedLM  Dependent Variable:  log_pop  
No. Observations:    1697     Method:              REML     
No. Groups:          94       Scale:               0.0502   
Min. group size:     6        Log-Likelihood:      -526.7063
Max. group size:     27       Converged:           Yes      
Mean group size:     18.1                                   
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept         66.688    0.987 67.598 0.000 64.754 68.622
after_treated      0.256    0.028  9.165 0.000  0.201  0.311
Group Var          0.051    8.732                           
Group x years Cov -0.000    0.034                           
years Var          0.001    0.001                           



In [13]:
# グループ変数を結合して単一の列にする
df['group'] = df['island_ids'].astype(str) + '_' + df['years'].astype(str)

# モデルの定義
model = mixedlm("log_pop ~ after_treated", df, groups=df["group"], re_formula="~after_treated + time_since_intervention")
result = model.fit()

# 結果の表示
print(result.summary())

/home/keisei/data-analysis/bridge-straw-effects/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/keisei/data-analysis/bridge-straw-effects/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/home/keisei/data-analysis/bridge-straw-effects/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/keisei/data-analysis/bridge-straw-effects/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/home/keisei/data-analysis/bridge-straw-effects/.

                           Mixed Linear Model Regression Results
Model:                         MixedLM            Dependent Variable:            log_pop   
No. Observations:              1697               Method:                        REML      
No. Groups:                    1697               Scale:                         0.2473    
Min. group size:               1                  Log-Likelihood:                -4259.0242
Max. group size:               1                  Converged:                     No        
Mean group size:               1.0                                                         
-------------------------------------------------------------------------------------------
                                            Coef.    Std.Err.      z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------
Intercept                                    4.554        0.039 118.178 0.000  4.479  4.630
after_treated  

/home/keisei/data-analysis/bridge-straw-effects/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
